In [80]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

from qrnn import get_model
from dataloader import get_data

months = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [47]:
data_set = 'Irish_2010'
year = '2010'
path = os.path.abspath(os.path.join(os.getcwd(), '../..'))

data = get_data(path, data_set)

100%|██████████| 918/918 [00:28<00:00, 32.04it/s]


In [96]:
month = 1
n_clusters = 2
method = 'hierarchical/euclidean'

path_cluster = os.path.join(path, 'result', data_set, 'clustering', 'point', method, f'n_clusters_{n_clusters}.csv')
clusters = pd.read_csv(path_cluster, header=None)

series = data[:, month-1, :months[month-1]*24]

In [75]:
# weather series
weather_48 = pd.read_csv(os.path.join(path, 'data', 'weather', f'weather_{data_set}_cleaned.csv'))['T'].values
weather_24 = []
for i in range(int(len(weather_48)/2)):
    weather_24.append((weather_48[i*2] + weather_48[i*2+1])/2)
weather_24 = np.array(weather_24)

weather = []
for i in range(12):
    weather.append(weather_24[sum(months[:i])*24:sum(months[:i+1])*24])

In [87]:
# Generate calendar series (week)
week = []
for i in range(months[month-1]):
    week_value = datetime.strptime(year + '%02d%02d' %(month, i+1), '%Y%m%d').weekday()
    for _ in range(24):
        week.append(week_value)
week = np.array(week) / 6   # normalization

In [94]:
# Generate calendar series (day)
day = []
for i in range(months[month-1]):
    for _ in range(24):
        day.append(_)
day = np.array(day) / 23   # normalization

In [125]:
for i in range(n_clusters):

    index = list(clusters[month-1] == i)
    sub_series = series[index]
    sub_series = np.sum(sub_series, axis=0)
    test = sub_series[-168:]
    train = sub_series[:-168]

    scale = np.zeros(2)
    scale[0] = np.max(train)
    scale[1] = np.min(train)
    train = (train - scale[1])/(scale[0] - scale[1])
    test = (test - scale[1])/(scale[0] - scale[1])
    
    lag = 24
    d = 1
    
    trainX, trainY = get_train_set_qrnn(train, weather[month-1], week, day, lag, d)
    
#     # Parameters
#     input_dim = 3
#     num_hidden_layers = 1
#     num_units = [200]
#     act = ['relu']
#     gauss_std = 0.3

#     # Get model
#     model = get_model(input_dim, num_units, act, gauss_std, num_hidden_layers)

#     # Train
#     model.fit(x=Xtrain, y=Ytrain, epochs=10)
    
    break

In [6]:
# # Generate the synthetic data
# x1 = np.sin(np.arange(0, 9, 0.01))
# x2 = np.cos(np.arange(0, 9, 0.01))
# x3 = x1**2
# x4 = (x1+x2)/2

# Xtrain = np.vstack((x2, x3, x4)).T
# Ytrain = np.array([x1]*99).T

In [124]:
def get_train_set_qrnn(data, temperature, week, day, lag, d):
    l = np.maximum(d * 24, lag)

    total_X = []
    total_Y = []
    for i in range(len(data) - l):

        elect = np.zeros(lag + d)
        elect[:lag] = data[i+l-lag:i+l]

        for j in range(d):
            elect[lag+j] = np.mean(data[i+l-(j+1)*24:i+l-j*24])

        temp = np.zeros(lag + 1 + d)
        temp[:lag+1] = temperature[i+l-lag:i+l+1]

        for j in range(d):
            temp[lag+j+1] = np.mean(temperature[i+l-(j+1)*24:i+l-j*24])

        X = np.hstack((elect, temp, week[i+l], day[i+l]))
        Y = data[i+l]
        total_X.append(X)
        total_Y.append(Y)

    total_X = np.array(total_X)
    total_Y = np.array(total_Y)
    
    return total_X, total_Y.reshape(-1, 1)